# 分析 final_solutions.csv ，通过代码作图，显示三维 Pareto Front

**独立代码块**

In [ ]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

data = np.loadtxt("final_solutions.csv", delimiter=",")

# data.shape = (N, 15)
# 假设前 12 列是决策变量 X，后 3 列是目标值 F。
X = data[:, :12]   # (N, 12)
F = data[:, 12:]   # (N, 3)

# 这里假设 F 里的三列已经是我们要画的数值(若需反转/还原，请在此之前做)
f0 = F[:, 0]
f1 = F[:, 1]
f2 = F[:, 2]

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(f0, f1, f2, c='b', marker='o', depthshade=False)

ax.set_xlabel("Objective 1 (F0)")
ax.set_ylabel("Objective 2 (F1)")
ax.set_zlabel("Objective 3 (F2)")
plt.title("Pareto Front in 3D")

plt.show()


# 读取 res_history 文件，分析代际间适应度优化的趋势
**独立代码块**

**以单目标为例: 逐代计算最小值、平均值**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

data = np.load("res_history.npz", allow_pickle=True)
final_X = data["final_X"]  # (n_nd, n_var), 最终非支配解/最优解
final_F = data["final_F"]  # (n_nd, n_obj)
history_F = data["history_F"]  # (n_gen,) 个元素, 每个元素 shape=(pop_size, n_obj)

# 打印信息:
print("final_X.shape:", final_X.shape)
print("final_F.shape:", final_F.shape)
print("历代数量 = ", len(history_F))

# 以单目标为例: 逐代计算最小值、平均值
best_per_gen = []
avg_per_gen = []
for gen_idx, F_gen in enumerate(history_F):
    best_val = np.min(F_gen[:, 1])   # 第1列目标(从0开始计数), shape=(pop_size,)
    avg_val  = np.mean(F_gen[:, 1])
    best_per_gen.append(best_val)
    avg_per_gen.append(avg_val)

# 绘制
gen_axis = np.arange(1, len(history_F)+1)
plt.figure()
plt.plot(gen_axis, best_per_gen, label="Best of Gen")
plt.plot(gen_axis, avg_per_gen, label="Mean of Gen")
plt.xlabel("Generation")
plt.ylabel("Objective Value")
plt.title("Convergence Over Generations")
plt.legend()
plt.show()


In [ ]:
# 加载存储的文件
data = np.load("res_history.npz", allow_pickle=True)
print(data.files)  # ['final_X', 'final_F', 'history_F']

final_X = data["final_X"]
final_F = data["final_F"]
history_F = data["history_F"]  # 这是一个 list，每一项是每代的 F

# 可以计算并绘制最优解、平均解的变化
import matplotlib.pyplot as plt
best_per_gen = []
avg_per_gen = []

for F_gen in history_F:
    best_per_gen.append(np.min(F_gen[:, 0]))  # 单目标最小值
    avg_per_gen.append(np.mean(F_gen[:, 0]))  # 单目标平均值

# 绘制收敛图
plt.figure()
plt.plot(range(1, len(history_F)+1), best_per_gen, label="Best of Gen")
plt.plot(range(1, len(history_F)+1), avg_per_gen, label="Average of Gen")
plt.xlabel("Generation")
plt.ylabel("Objective Value")
plt.title("Convergence Over Generations")
plt.legend()
plt.show()


# 每一代（generation）和非支配解（nondominated solutions）文件，分析判断算法的优化趋势与收敛性
**独立代码块，需要自定义chkpt_dir与n_gen**

In [ ]:
%matplotlib widget
import numpy as np
import math
import os
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# 指定一个支持中文的字体，比如 Windows 上常见的 "SimHei" (黑体)
matplotlib.rcParams["font.sans-serif"] = ["SimHei"]  
matplotlib.rcParams["axes.unicode_minus"] = False    # 正常显示负号
from pymoo.indicators.hv import HV

chkpt_dir = r"F:\ResearchMainStream\0.ResearchBySection\C.动力学模型\参数优化\参数优化实现\ParallelSweepSimpack\结果分析组\0210早-nsga2-180群150代-收敛"  # 保存 generation_xxx.npz 的目录
n_gen = 150            # 总迭代轮次
n_obj = 3             # 你的目标函数个数

# 存储每一代的整个种群目标值（适应度函数）
all_F_history = []  # 这是一个列表，元素为 F_gen 数组 (pop_size, n_obj)

for gen in range(1, n_gen+1):
    filename = os.path.join(chkpt_dir, f"generation_{gen}.npz")
    data = np.load(filename)
    F_gen = data["F"]  # 形如 (pop_size, n_obj)
    all_F_history.append(F_gen)

# 现在 all_F_history[0] 对应第1代的所有目标值 (pop_size, n_obj),
# all_F_history[1] 对应第2代的所有目标值, ... 依次类推。

# 对于每一代，计算 F_gen 在所有个体上的最小值(逐个目标)
best_per_gen = np.zeros((n_gen, n_obj))  # shape = (88, 3)

for i in range(n_gen):
    F_gen = all_F_history[i]  # (pop_size, n_obj)
    # 计算 3 个目标各自的最小值
    best_per_gen[i, :] = F_gen.min(axis=0)

# 绘制收敛图
plt.figure(figsize=(6,4))
gen_axis = np.arange(1, n_gen+1)  # x轴为第1到第88代
for j in range(n_obj):
    plt.plot(gen_axis, best_per_gen[:, j], label=f"F{j}的最小值")
plt.xlabel("迭代代数 (generation)")
plt.ylabel("目标函数值 (Objective value)")
plt.title("每代最优目标值的变化曲线")
plt.legend()
plt.grid(True)
plt.show()

# 每一代，计算 F_gen 平均值
avg_per_gen = np.zeros((n_gen, n_obj))  # 用于存储每一代 3 个目标的平均值

for i in range(n_gen):
    F_gen = all_F_history[i]
    avg_per_gen[i, :] = F_gen.mean(axis=0)  # 计算该代目标值的均值

# 绘制平均值随迭代的变化
gen_axis = np.arange(1, n_gen + 1)  # [1, 2, ..., 88]
plt.figure(figsize=(6,4))
for j in range(n_obj):
    plt.plot(gen_axis, avg_per_gen[:, j], label=f"F{j}平均值")

plt.xlabel("迭代代数 (generation)")
plt.ylabel("目标函数平均值")
plt.title("每代目标平均值趋势")
plt.legend()
plt.grid(True)
plt.show()


fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111, projection='3d')

for gen in [100, 150]:  # 只示例第1代和第88代
    filename_nd = os.path.join(chkpt_dir, f"generation_{gen}_nondom.npz")
    data_nd = np.load(filename_nd)
    F_nd = data_nd["F"]  # (n_nd, n_obj) 这里 n_obj=3
    ax.scatter(F_nd[:,0], F_nd[:,1], F_nd[:,2], depthshade=False,  label=f"Gen {gen} ND")

ax.set_xlabel("目标1 (F0)")
ax.set_ylabel("目标2 (F1)")
ax.set_zlabel("目标3 (F2)")
ax.legend()
ax.set_title("各代非支配解的 3D 散点分布")
plt.show()

# 高维指标：超体积 (Hypervolume) 或 IGD
# 在多目标优化中，衡量优化质量和收敛性的常用指标还包括 超体积（hypervolume, HV）、GD/IGD（Generational Distance / Inverted Generational Distance）等。
# 通过计算每一代的非支配解的超体积，可以看到随着迭代次数增加，超体积是否单调增加并逐渐收敛。
# 这类指标能够更全面地衡量多目标收敛和分布情况，尤其是当目标数大于 2 或 3 时更为重要。
# 如果需要计算超体积，pymoo 自带了一些指标工具，可以参考 pymoo文档 或者自行使用常见的 HV 算法包。

MAXoverUB_F1 = 0
MAXoverUB_F2 = 1000
MAXoverUB_F3 = math.sqrt(3 * 3 + 3 * 3) * 100 # 由约束定义上限

# 设定一个参考点 (reference point)，大于或等于所有解的最大目标值
ref_point = np.array([MAXoverUB_F1, MAXoverUB_F2, MAXoverUB_F3])

hv_indicator = HV(ref_point=ref_point)

hv_values = []
for gen in range(1, n_gen+1):
    filename_nd = os.path.join(chkpt_dir, f"generation_{gen}_nondom.npz")
    data_nd = np.load(filename_nd)
    F_nd = data_nd["F"]  # (n_nd, 3)
    hv = hv_indicator.do(F_nd)
    hv_values.append(hv)

# hv_values[i] 就是第 i 代的超体积
fig = plt.figure()
plt.plot(range(1, n_gen+1), hv_values, marker='o')
plt.xlabel("迭代代数 (generation)")
plt.ylabel("Hypervolume")
plt.title("超体积收敛曲线")
plt.show()

# 将 NSGA-2 和 NSGA-3 的非支配解合并，重新计算它们在一起时的整体非支配前沿（ND），并用不同颜色区分它们各自的解

**独立代码块，需要自定义chkpt_dir**

文件名按照以下方式重命名：
 - generation_2_nondom.npz 存放 NSGA-2 第 2 代的非支配解
 - generation_3_nondom.npz 存放 NSGA-3 第 3 代的非支配解
 - 将一个 generation_1_nondom.npz 作为 NSGA-2 第 1 代的非支配解（示例中如果不需要，可以自行去掉此部分）

In [ ]:
%matplotlib widget

import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pymoo.util.nds.non_dominated_sorting import NonDominatedSorting
# 指定一个支持中文的字体，比如 Windows 上常见的 "SimHei" (黑体)
matplotlib.rcParams["font.sans-serif"] = ["SimHei"]  
matplotlib.rcParams["axes.unicode_minus"] = False    # 正常显示负号

chkpt_dir = r"F:\ResearchMainStream\0.ResearchBySection\C.动力学模型\参数优化\参数优化实现\ParallelSweepSimpack\结果分析组\NSGA23算法效能对比"

# =========== 1. 读取 NSGA-2 / NSGA-3 各自的非支配解 ===========
# 假设:
#   - generation_1_nondom.npz 是 NSGA-2 第 1 代的非支配解
#   - generation_2_nondom.npz 是 NSGA-2 第 2 代的非支配解
#   - generation_3_nondom.npz 是 NSGA-3 第 3 代的非支配解
# 如果你只想对比某几代，也可以只读那几份文件。

# ---------- 读取 NSGA-2 的非支配解 ----------
filename_nsga2_gen1 = os.path.join(chkpt_dir, "generation_1_nondom.npz")
data_nsga2_gen1 = np.load(filename_nsga2_gen1)
F_nsga2_gen1 = data_nsga2_gen1["F"]  # (n_nd1, n_obj)
X_nsga2_gen1 = data_nsga2_gen1["X"]

filename_nsga2_gen2 = os.path.join(chkpt_dir, "generation_2_nondom.npz")
data_nsga2_gen2 = np.load(filename_nsga2_gen2)
F_nsga2_gen2 = data_nsga2_gen2["F"]  # (n_nd2, n_obj)
X_nsga2_gen2 = data_nsga2_gen2["X"]

# ---------- 读取 NSGA-3 的非支配解 ----------
filename_nsga3_gen3 = os.path.join(chkpt_dir, "generation_3_nondom.npz")
data_nsga3_gen3 = np.load(filename_nsga3_gen3)
F_nsga3_gen3 = data_nsga3_gen3["F"]  # (n_nd3, n_obj)
X_nsga3_gen3 = data_nsga3_gen3["X"]

# =========== 2. 合并解并标记其来源 (NSGA-2 or NSGA-3) ===========
# 假设 n_obj = 3
F_all = np.concatenate([F_nsga2_gen1, F_nsga2_gen2, F_nsga3_gen3], axis=0)
X_all = np.concatenate([X_nsga2_gen1, X_nsga2_gen2, X_nsga3_gen3], axis=0)

# 给每个解打一个标签，用于区分它是 NSGA-2 或 NSGA-3：
#  - 这里用数字 2 表示 NSGA-2，数字 3 表示 NSGA-3。
labels_nsga2_gen1 = np.array([2]*len(F_nsga2_gen1))
labels_nsga2_gen2 = np.array([2]*len(F_nsga2_gen2))
labels_nsga3_gen3 = np.array([3]*len(F_nsga3_gen3))

labels_all = np.concatenate([labels_nsga2_gen1, labels_nsga2_gen2, labels_nsga3_gen3], axis=0)

# =========== 3. 对合并后的所有解做一次新的非支配排序 ===========
nd_front = NonDominatedSorting().do(F_all, only_non_dominated_front=True)

# nd_front 是索引数组, 指示了合并后的 F_all 中哪些是非支配解
F_nd_all = F_all[nd_front]
X_nd_all = X_all[nd_front]
labels_nd_all = labels_all[nd_front]  # 同步提取它们的来源标签

# =========== 4. 分别取出 NSGA-2 和 NSGA-3 在此合并 ND 中的解 ===========
mask_nsga2 = (labels_nd_all == 2)
mask_nsga3 = (labels_nd_all == 3)

F_nd_nsga2 = F_nd_all[mask_nsga2]
F_nd_nsga3 = F_nd_all[mask_nsga3]

# =========== 5. 绘制散点图 ===========

fig = plt.figure(figsize=(7, 5))
ax = fig.add_subplot(111, projection='3d')

# 为了让效果更明显，这里演示一个简单的坐标变换
# 例如将 F0 用 -F0*3.6（负号是为了让“越大越好”排布更直观，或根据自己需求修正），
# F1 不变，F2 做除以200 等等
def transform(F):
    return (-F[:, 0]*3.6, F[:, 1], F[:, 2]/200)

# 绘制 NSGA-2（合并后ND）
if len(F_nd_nsga2) > 0:
    x2, y2, z2 = transform(F_nd_nsga2)
    ax.scatter(x2, y2, z2, c='red', marker='o', label='NSGA-2 ND')

# 绘制 NSGA-3（合并后ND）
if len(F_nd_nsga3) > 0:
    x3, y3, z3 = transform(F_nd_nsga3)
    ax.scatter(x3, y3, z3, c='blue', marker='^', label='NSGA-3 ND')

ax.set_xlabel("临界速度 km/h (F0)")
ax.set_ylabel("磨耗数 (F1)")
ax.set_zlabel("Sperling指标 (F2)")
ax.set_title("合并后非支配解 (ND) 的 3D 散点分布比较")
ax.legend()
plt.show()


# Markdown 占位

In [5]:
# 代码占位